In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install -U langchain-community

In [ ]:
pip install chromadb

In [ ]:
pip install pyngrok

# RAG-pipeline: Qwen2.5-0.5B+hybrid_search+reranker

In [ ]:
import os
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Extra
from typing import Optional, List, Any

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseRetriever, Document
from langchain.retrievers import EnsembleRetriever

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
import torch.nn.functional as F

# === LLM ===
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.01)

SYSTEM_PROMPT = (
    "Ты помощник по документации Moodle. "
    "Отвечай кратко, точно и по теме. "
    "Если ответа нет в контексте, используй общие знания и постарайся помочь. "
    "Отвечай на том языке, на котором задан вопрос.\n\n"
)

class LocalLLM(LLM):
    class Config:
        extra = Extra.allow

    def __init__(self, pipeline: Any):
        super().__init__()
        self.__dict__["pipeline"] = pipeline

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        try:
            full_prompt = SYSTEM_PROMPT + prompt
            output = self.pipeline(full_prompt)[0]["generated_text"]
            return output[len(prompt):]
        except Exception:
            return "Произошла ошибка при генерации ответа."

    @property
    def _llm_type(self) -> str:
        return "local_llm"

# === Embeddings ===
embedding_model = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

# === ChromaDB ===
chroma_path = "/content/drive/MyDrive/RAG_Moodle/chroma_db_qwen3"
db = Chroma(persist_directory=chroma_path, embedding_function=embedding_model)

# === Memory ===
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# === BGE reranker ===
reranker_name = "BAAI/bge-reranker-v2-m3"
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_name)
reranker_model = AutoModel.from_pretrained(reranker_name)

def rerank(query: str, documents: List[str], top_n: int = 8) -> List[str]:
    try:
        if not documents:
            return []

        pairs = [(query, doc) for doc in documents]
        inputs = reranker_tokenizer(pairs, padding=True, truncation=True, return_tensors="pt")

        with torch.no_grad():
            outputs = reranker_model(**inputs)
            if hasattr(outputs, "logits"):
                scores = outputs.logits.view(-1)
            else:
                scores = outputs.last_hidden_state[:, 0, :].mean(dim=1)

        top_indices = torch.topk(scores, k=min(top_n, len(documents))).indices.tolist()
        return [documents[i] for i in top_indices]
    except Exception:
        return documents[:top_n]

# === Кастомный RAG Retriever с rerank ===
class RerankRetriever(BaseRetriever):
    def __init__(self, base_retriever, reranker_model, reranker_tokenizer, top_k=8):
        super().__init__()
        object.__setattr__(self, "base_retriever", base_retriever)
        object.__setattr__(self, "reranker_model", reranker_model)
        object.__setattr__(self, "reranker_tokenizer", reranker_tokenizer)
        object.__setattr__(self, "top_k", top_k)

    def get_relevant_documents(self, query: str) -> List[Document]:
        try:
            docs = self.base_retriever.get_relevant_documents(query)
            texts = [doc.page_content for doc in docs]
            top_texts = rerank(query, texts, top_n=self.top_k)
            return [doc for doc in docs if doc.page_content in top_texts]
        except Exception:
            return []


# === Гибридный Retriever ===
vector_retriever = db.as_retriever(search_kwargs={"k": 8})
keyword_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 8})

hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, keyword_retriever],
    weights=[0.5, 0.5]
)

# === RAG с rerank ===
retriever_with_rerank = RerankRetriever(
    base_retriever=hybrid_retriever,
    reranker_model=reranker_model,
    reranker_tokenizer=reranker_tokenizer,
    top_k=8
)

local_llm = LocalLLM(pipeline=llm_pipeline)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=local_llm,
    retriever=retriever_with_rerank,
    memory=memory
)


In [ ]:
import os
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Extra
from typing import Optional, List, Any

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseRetriever, Document
from langchain.retrievers import EnsembleRetriever

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
import torch.nn.functional as F

# === LLM ===
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.01)

SYSTEM_PROMPT = (
    "Ты помощник по документации Moodle. "
    "Отвечай кратко, точно и по теме. "
    "Если ответа нет в контексте, используй общие знания и постарайся помочь. "
    "Отвечай на том языке, на котором задан вопрос.\n\n"
)

class LocalLLM(LLM):
    class Config:
        extra = Extra.allow

    def __init__(self, pipeline: Any):
        super().__init__()
        self.__dict__["pipeline"] = pipeline

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        try:
            full_prompt = SYSTEM_PROMPT + prompt
            output = self.pipeline(full_prompt)[0]["generated_text"]
            return output[len(prompt):]
        except Exception:
            return "Произошла ошибка при генерации ответа."

    @property
    def _llm_type(self) -> str:
        return "local_llm"

# === Embeddings ===
embedding_model = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

# === ChromaDB ===
chroma_path = "/content/drive/MyDrive/RAG_Moodle/chroma_db_qwen3"
db = Chroma(persist_directory=chroma_path, embedding_function=embedding_model)

# === Memory ===
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# === BGE reranker ===
reranker_name = "BAAI/bge-reranker-v2-m3"
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_name)
reranker_model = AutoModel.from_pretrained(reranker_name)

def rerank(query: str, documents: List[str], top_n: int = 8) -> List[str]:
    try:
        if not documents:
            return []

        pairs = [(query, doc) for doc in documents]
        inputs = reranker_tokenizer(pairs, padding=True, truncation=True, return_tensors="pt")

        with torch.no_grad():
            outputs = reranker_model(**inputs)
            if hasattr(outputs, "logits"):
                scores = outputs.logits.view(-1)
            else:
                scores = outputs.last_hidden_state[:, 0, :].mean(dim=1)

        top_indices = torch.topk(scores, k=min(top_n, len(documents))).indices.tolist()
        return [documents[i] for i in top_indices]
    except Exception:
        return documents[:top_n]

# === Кастомный RAG Retriever с rerank ===
class RerankRetriever(BaseRetriever):
    def __init__(self, base_retriever, reranker_model, reranker_tokenizer, top_k=8):
        super().__init__()
        object.__setattr__(self, "base_retriever", base_retriever)
        object.__setattr__(self, "reranker_model", reranker_model)
        object.__setattr__(self, "reranker_tokenizer", reranker_tokenizer)
        object.__setattr__(self, "top_k", top_k)

    def get_relevant_documents(self, query: str) -> List[Document]:
        try:
            docs = self.base_retriever.get_relevant_documents(query)
            texts = [doc.page_content for doc in docs]
            top_texts = rerank(query, texts, top_n=self.top_k)
            return [doc for doc in docs if doc.page_content in top_texts]
        except Exception:
            return []


# === Гибридный Retriever ===
vector_retriever = db.as_retriever(search_kwargs={"k": 8})
keyword_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 8})

hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, keyword_retriever],
    weights=[0.5, 0.5]
)

# === RAG с rerank ===
retriever_with_rerank = RerankRetriever(
    base_retriever=hybrid_retriever,
    reranker_model=reranker_model,
    reranker_tokenizer=reranker_tokenizer,
    top_k=8
)

local_llm = LocalLLM(pipeline=llm_pipeline)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=local_llm,
    retriever=retriever_with_rerank,
    memory=memory
)



# FastAPI - приложение

In [ ]:
from fastapi import FastAPI
from pyngrok import ngrok, conf
import nest_asyncio
import uvicorn
import os

# Вставь сюда свой токен
conf.get_default().auth_token = "ВАШ_ТОКЕН"

# === FastAPI ===
app = FastAPI()

class Question(BaseModel):
    query: str

@app.post("/ask")
def ask_q(input: Question):
    try:
        if not input.query or input.query.strip() == "":
            raise HTTPException(status_code=400, detail="Запрос не может быть пустым.")

        result = qa_chain.run(input.query)
        return {"answer": result}

    except HTTPException as http_err:
        raise http_err

    except Exception:
        return JSONResponse(
            status_code=500,
            content={"error": "Произошла ошибка при генерации ответа."}
        )


# Подключение ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Запуск сервера
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

# Локальный вывод

In [ ]:
result1 = qa_chain.run("Как создать новый курс в Moodle?")
print(result1)

In [ ]:
result2 = qa_chain.run("Как настроить систему оценок в Moodle?")
print(result2)

In [ ]:
result3 = qa_chain.run("Как просмотреть журналы активности пользователей?")
print(result3)

### часть экспериментов ретривером

In [ ]:
#2.5 - 0.5В+reranker_m3+embed_qwen5+chunks256+hybrid
result = qa_chain.run("Как создать новый курс в Moodle?")
print(result)

In [ ]:
#2.5 - 0.5В+reranker_m3+embed_qwen5+chunks256+hybrid
result4 = qa_chain.run("Как настроить систему оценок в Moodle?")
print(result4)